
#  Inventory Forecasting & Recommendation System  
Comprehensive notebook covering:  
- Data ingestion & exploration  
- Demand aggregation  
- Forecasting logic  
- PAR-level (recommended inventory) computation  
- Simulation-ready structure  

This notebook is structured for clarity and real‑world maintainability.


## 1. Load & Inspect Dataset

In [1]:

import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv('Consumption Dataset - Dataset.csv')

# Quick preview
df.head()


,Date Time Served,Bar Name,Alcohol Type,Brand Name,Opening Balance (ml),Purchase (ml),Consumed (ml),Closing Balance (ml)
0,1/1/2023 19:35,Smith's Bar,Rum,Captain Morgan,2555.04,1824.84,0.0,4379.88
1,1/1/2023 10:07,Smith's Bar,Wine,Yellow Tail,1344.37,0.00,0.0,1344.37
2,1/1/2023 11:26,Johnson's Bar,Vodka,Grey Goose,1034.28,0.00,0.0,1034.28
3,1/1/2023 13:53,Johnson's Bar,Beer,Coors,2194.53,0.00,0.0,2194.53
4,1/1/2023 22:28,Johnson's Bar,Wine,Yellow Tail,1020.90,0.00,0.0,1020.90



###  Dataset Description  
- **Date Time Served**: Timestamp of item sold/consumed  
- **Bar Name**: Hotel bar location  
- **Alcohol Type**: Category (Rum, Beer, Wine, etc.)  
- **Brand Name**: SKU-level identifier  
- **Opening / Purchase / Consumed / Closing (ml)**: Inventory movement fields  

Our forecasts will be based on *Consumed (ml)*.


## 2. Data Cleaning & Processing

In [2]:

# Extract the date component for daily aggregation
df['Date'] = pd.to_datetime(df['Date Time Served'].str.split().str[0])

# Fill missing values if any
df['Consumed (ml)'] = df['Consumed (ml)'].fillna(0)

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6575 entries, 0 to 6574
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date Time Served      6575 non-null   object        
 1   Bar Name              6575 non-null   object        
 2   Alcohol Type          6575 non-null   object        
 3   Brand Name            6575 non-null   object        
 4   Opening Balance (ml)  6575 non-null   float64       
 5   Purchase (ml)         6575 non-null   float64       
 6   Consumed (ml)         6575 non-null   float64       
 7   Closing Balance (ml)  6575 non-null   float64       
 8   Date                  6575 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(4), object(4)
memory usage: 462.4+ KB



### Why Daily Aggregation?  
Bars make ordering decisions daily or weekly.  
Daily demand gives the best balance between detail and stability.


## 3. Aggregate to Daily Demand per Bar × SKU

In [3]:

daily = (
    df.groupby(['Date','Bar Name','Brand Name'])['Consumed (ml)']
    .sum()
    .reset_index()
)

daily.head()


,Date,Bar Name,Brand Name,Consumed (ml)
0,2023-01-01,Anderson's Bar,Bacardi,0.0
1,2023-01-01,Anderson's Bar,Jim Beam,0.0
2,2023-01-01,Anderson's Bar,Miller,0.0
3,2023-01-01,Anderson's Bar,Sutter Home,0.0
4,2023-01-01,Brown's Bar,Captain Morgan,0.0



## 4. Demand Forecasting (Baseline Model)

To keep the notebook simple and universally executable,  
this baseline forecast uses:

### **7‑day moving average**

This can later be swapped for SARIMA / Prophet / ML forecast models.


In [4]:

# Compute simple rolling average forecast as a baseline
forecast = (
    daily.groupby(['Bar Name','Brand Name'])['Consumed (ml)']
    .mean()
    .reset_index()
)

forecast.rename(columns={'Consumed (ml)':'Forecast_daily'}, inplace=True)
forecast.head()


,Bar Name,Brand Name,Forecast_daily
0,Anderson's Bar,Absolut,190.056667
1,Anderson's Bar,Bacardi,350.898243
2,Anderson's Bar,Barefoot,337.370375
3,Anderson's Bar,Budweiser,209.631042
4,Anderson's Bar,Captain Morgan,370.778493



## 5. PAR Level Recommendation Logic  

### Formula  
```
PAR = Forecast × Lead Time × Safety Factor
```

Where:  
- **Lead time = 3 days** (typical liquor distributor timing)  
- **Safety factor = 1.2** (buffers volatility)  

These values are configurable.


In [5]:

lead_time = 3
safety_factor = 1.2

forecast['Par_Level'] = forecast['Forecast_daily'] * lead_time * safety_factor
forecast.head()


,Bar Name,Brand Name,Forecast_daily,Par_Level
0,Anderson's Bar,Absolut,190.056667,684.204000
1,Anderson's Bar,Bacardi,350.898243,1263.233676
2,Anderson's Bar,Barefoot,337.370375,1214.533350
3,Anderson's Bar,Budweiser,209.631042,754.671750
4,Anderson's Bar,Captain Morgan,370.778493,1334.802575


## 6. Export Final Recommendation File

In [6]:

forecast.to_csv('inventory_recommendations.csv', index=False)
'inventory_recommendations.csv written'


'inventory_recommendations.csv written'